In [1]:
from __future__ import print_function
import logging

import grpc

import question_pb2
import question_pb2_grpc

In [2]:
from concurrent import futures


import contest_pb2
import contest_pb2_grpc

import time
import re

import numpy as np
import pandas as pd
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
def initialize():
    
    '''
    Contest
    :stub.login: 接口用于登陆
    :return stub: 和远程主机建立的Contest连接
    :return response.session_key: 远程主机返回的session_key，用于后续的答案提交
    '''
    channel = grpc.insecure_channel('47.103.23.116:56702')
    stub = contest_pb2_grpc.ContestStub(channel)
    response = stub.login(contest_pb2.LoginRequest(user_id = 69, user_pin ='vKfPGNrn'))
    print(response)
    return stub,response.session_key

In [5]:
def get_data(_sequence,stub2):
    '''
    从远程主机获取某一天的市场数据
    :param _sequence: 某一天。'0'代表远程主机最新的数据，之后依次加1。注：不支持请求历史数据
    :param stub2: 和远程主机建立的Question连接
    :return response2: 
    :       .sequnence: 当前sequence号，之后的请求在此基础上+1
    :       .capital: 选手当前可用资金
    :       .dailystk: 当天所有的股票数据。第一维是各支股票。
                       每只股票内的数据顺序为[天数、股票代码、开盘价、最高价、最低价、收盘价、成交量]
    '''
    response2 = stub2.get_question(question_pb2.QuestionRequest(user_id = 69, sequence = _sequence))
    return response2

In [58]:
'''
用来储存所有从远程主机返回的数据（包括历史数据）
'''
all_dic = pd.DataFrame(columns = ['day','stock','open','high','low','close','volume'])

In [6]:
channel2 = grpc.insecure_channel('47.103.23.116:56701')
stub2 = question_pb2_grpc.QuestionStub(channel2)
my_sequence = 0

response = get_data(my_sequence, stub2)
response

user_id: 69
sequence: 176
has_next_question: true
capital: 594643313.345914
dailystk {
  values: 176.0
  values: 1000.0
  values: 9.98316
  values: 10.19304
  values: 9.98316
  values: 10.04586
  values: 16080.0
}
dailystk {
  values: 176.0
  values: 1001.0
  values: 17.49
  values: 17.49
  values: 17.49
  values: 17.49
  values: 0.0
}
dailystk {
  values: 176.0
  values: 1002.0
  values: 52.088519999999995
  values: 52.088519999999995
  values: 51.117659999999994
  values: 51.37638
  values: 67968.0
}
dailystk {
  values: 176.0
  values: 1003.0
  values: 6.1644
  values: 6.1644
  values: 6.1644
  values: 6.1644
  values: 0.0
}
dailystk {
  values: 176.0
  values: 1004.0
  values: 185.658
  values: 186.91199999999998
  values: 185.658
  values: 186.91199999999998
  values: 2400.0
}
dailystk {
  values: 176.0
  values: 1005.0
  values: 62.105999999999995
  values: 62.105999999999995
  values: 62.105999999999995
  values: 62.105999999999995
  values: 0.0
}
dailystk {
  values: 176.0
  va

In [42]:
my_sequence = response.sequence
my_sequence

176

In [41]:
len(response.dailystk)

351

In [40]:
response.dailystk[0].values

[176.0, 1000.0, 9.98316, 10.19304, 9.98316, 10.04586, 16080.0]

In [59]:
temp_dic = pd.DataFrame(columns = ['day','stock','open','high','low','close','volume'])


for i in range(0,len(response.dailystk)):
    
    responseData = response.dailystk[i].values
    temp_dic = temp_dic.append({'day':responseData[0],'stock':int(responseData[1]), 'open':responseData[2],'high':responseData[3],'low':responseData[4],'close':responseData[5],'volume':responseData[6]}, ignore_index=True)

all_dic = all_dic.append(temp_dic) # 这样更节省时间

In [60]:
all_dic

,day,stock,open,high,low,close,volume
0,176.0,1000.0,9.98316,10.19304,9.98316,10.04586,16080.0
1,176.0,1001.0,17.49000,17.49000,17.49000,17.49000,0.0
2,176.0,1002.0,52.08852,52.08852,51.11766,51.37638,67968.0
3,176.0,1003.0,6.16440,6.16440,6.16440,6.16440,0.0
4,176.0,1004.0,185.65800,186.91200,185.65800,186.91200,2400.0
...,...,...,...,...,...,...,...
346,176.0,1346.0,68.12586,68.12586,68.12586,68.12586,0.0
347,176.0,1347.0,22.30800,22.77000,22.30800,22.77000,0.0
348,176.0,1348.0,166.51800,166.51800,161.50200,161.50200,9120.0
349,176.0,1349.0,54.71400,54.78000,53.46000,53.46000,48960.0


In [62]:
set([1, 2, 1, 2])

{1, 2}

In [63]:
# 把全部数据分拆到股票各自的列表里。需要什么指标的话在这里直接算，节省时间。
stocklist = list(set(all_dic['stock']))
dic = {}
for i in stocklist:
    j = all_dic[all_dic['stock']==i]
    j.index = j['day']
    j['zhangfu'] = j['close'].diff()/j['close'].shift() # 类似这样的指标什么的
    j['100dayRS'] = j['close'].diff(100)/j['close'].shift(100)
    dic[i] = j

In [75]:
dic[1000]

,day,stock,open,high,low,close,volume,zhangfu,100dayRS
day,,,,,,,,,
176.0,176.0,1000.0,9.98316,10.19304,9.98316,10.04586,16080.0,NaN,NaN


In [67]:
all_dic[all_dic['stock']==stocklist[0]]

,day,stock,open,high,low,close,volume
0,176.0,1000.0,9.98316,10.19304,9.98316,10.04586,16080.0


In [76]:
df = pd.DataFrame(np.arange(24).reshape(6,4),index=['r1','r2','r3','r4','r5','r6'],columns=['A','B','C','D'])
df

,A,B,C,D
r1,0,1,2,3
r2,4,5,6,7
r3,8,9,10,11
r4,12,13,14,15
r5,16,17,18,19
r6,20,21,22,23


In [77]:
df.shift()

,A,B,C,D
r1,NaN,NaN,NaN,NaN
r2,0.0,1.0,2.0,3.0
r3,4.0,5.0,6.0,7.0
r4,8.0,9.0,10.0,11.0
r5,12.0,13.0,14.0,15.0
r6,16.0,17.0,18.0,19.0


In [80]:
df.diff(periods=2)

,A,B,C,D
r1,NaN,NaN,NaN,NaN
r2,NaN,NaN,NaN,NaN
r3,8.0,8.0,8.0,8.0
r4,8.0,8.0,8.0,8.0
r5,8.0,8.0,8.0,8.0
r6,8.0,8.0,8.0,8.0


In [83]:
pd.Series([1, 3, 4, 2, 1])

0    1
1    3
2    4
3    2
4    1
dtype: int64

In [84]:
pd.Series([1, 3, 4, 2, 1]).rank()

0    1.5
1    4.0
2    5.0
3    3.0
4    1.5
dtype: float64

In [5]:
if 1!=-1 and 1!=0:
    print(1)

1


In [24]:
a = [i for i in range(0, 5)]

In [25]:
a

[0, 1, 2, 3, 4]

In [31]:
shifen = {}

In [28]:
shifen[0] = 1

In [32]:
shifen[0].append(a)

KeyError: 0

In [33]:
shifen

{}

In [34]:
for i in [0, 9]:
    print(i)

0
9


In [35]:
a = pd.read_csv('./factor/CONTEST_DATA_IN_SAMPLE_2.csv',names=['day','stock','open','high','low','close','volume'])

In [41]:
a.head()

,day,stock,open,high,low,close,volume
0,0,1000,15.76278,15.99378,15.75882,15.75882,103776
1,0,1001,200.31000,202.63320,199.62360,200.97000,6720
2,0,1002,81.22554,82.88808,81.22554,82.88808,528
3,0,1003,15.61560,15.61560,15.61560,15.61560,0
4,0,1004,313.39440,315.36120,310.20660,315.36120,35184


In [39]:
j = a[a['stock'] == 1000]
j

,day,stock,open,high,low,close,volume
0,0,1000,15.76278,15.99378,15.75882,15.75882,103776
351,1,1000,15.67038,15.75024,15.61956,15.61956,8032
702,2,1000,15.34236,15.39714,15.32124,15.39714,59440
1053,3,1000,15.51924,15.77928,15.51924,15.77928,43264
1404,4,1000,15.72912,15.72912,14.87574,14.87574,90944
...,...,...,...,...,...,...,...
279045,795,1000,54.56022,55.72446,54.56022,55.41756,77120
279396,796,1000,54.81234,55.06446,54.22428,54.22428,71520
279747,797,1000,56.79300,58.08660,53.69100,55.57860,241392
280098,798,1000,53.72400,56.00100,53.72400,54.78660,295920


In [40]:
j['zhangfu'] = j['close'].diff()/j['close'].shift()
j

<ipython-input-40-c0ea371f224b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  j['zhangfu'] = j['close'].diff()/j['close'].shift()


,day,stock,open,high,low,close,volume,zhangfu
0,0,1000,15.76278,15.99378,15.75882,15.75882,103776,NaN
351,1,1000,15.67038,15.75024,15.61956,15.61956,8032,-0.008837
702,2,1000,15.34236,15.39714,15.32124,15.39714,59440,-0.014240
1053,3,1000,15.51924,15.77928,15.51924,15.77928,43264,0.024819
1404,4,1000,15.72912,15.72912,14.87574,14.87574,90944,-0.057261
...,...,...,...,...,...,...,...,...
279045,795,1000,54.56022,55.72446,54.56022,55.41756,77120,0.011042
279396,796,1000,54.81234,55.06446,54.22428,54.22428,71520,-0.021533
279747,797,1000,56.79300,58.08660,53.69100,55.57860,241392,0.024976
280098,798,1000,53.72400,56.00100,53.72400,54.78660,295920,-0.014250


In [42]:
j.rolling(6).max()

,day,stock,open,high,low,close,volume,zhangfu
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
279045,795.0,1000.0,54.56022,55.72446,54.56022,55.41756,105296.0,0.038434
279396,796.0,1000.0,54.81234,55.72446,54.56022,55.41756,105296.0,0.038434
279747,797.0,1000.0,56.79300,58.08660,54.56022,55.57860,241392.0,0.038434
280098,798.0,1000.0,56.79300,58.08660,54.56022,55.57860,295920.0,0.038434
